In [18]:
import sys
sys.path.append("..")
import mindspore
from mindspore import context
from mindspore import nn
from mindspore import Tensor
from mindspore import ops
from src.bert import BertForPretraining
from src.config import BertConfig
from src.tokenizer import BertTokenizer
from model import BertBinaryClassificationModel
from mindnlp.mindnlp.mindtext.dataset.classification.sst import SST2Dataset


In [19]:
context.set_context(mode=context.GRAPH_MODE)
#context.set_context(mode=context.PYNATIVE_MODE)


dataset2 = SST2Dataset(paths='SST-2',
                      tokenizer="bert-base-uncased",
                      max_length=512,
                      truncation_strategy=True,
                      columns_list=['input_ids', 'token_type_ids', 'attention_mask', 'label'],
                      test_columns_list=['input_ids', 'token_type_ids', 'attention_mask'],
                      batch_size=1)

sst_2_ds2 = dataset2()

SST-2 dataset loadding: 100%|██████████| 67349/67349 [00:04<00:00, 16717.60it/s] 
SST-2 train dataset sentence preprocess bar(tokenize).: 100%|██████████| 67349/67349 [00:23<00:00, 2823.69it/s]
SST-2 train dataset processing.: 100%|██████████| 67349/67349 [00:03<00:00, 17073.86it/s] 
[WARNING] ME(3260:140362055714560,MainProcess):2022-11-26-13:36:57.612.677 [mindspore/dataset/core/validator_helpers.py:806] 'TypeCast' from mindspore.dataset.transforms.c_transforms is deprecated from version 1.8 and will be removed in a future version. Use 'TypeCast' from mindspore.dataset.transforms instead.
Writing data to .mindrecord file: 100%|██████████| 67349/67349 [03:10<00:00, 353.66it/s]
SST-2 dev dataset sentence preprocess bar(tokenize).: 100%|██████████| 872/872 [00:00<00:00, 1875.74it/s]
SST-2 dev dataset processing.: 100%|██████████| 872/872 [00:00<00:00, 16211.02it/s]
[WARNING] ME(3260:140362055714560,MainProcess):2022-11-26-13:40:09.467.863 [mindspore/dataset/core/validator_helpers.py:806

In [20]:

train_dataset = sst_2_ds2['train']
dev_dataset = sst_2_ds2['dev']
print(train_dataset.get_dataset_size())
print(train_dataset.get_batch_size())

print(dev_dataset.get_dataset_size())
print(dev_dataset.get_batch_size())

67349
1
872
1


In [21]:
# 保存mindrecord
train_dataset.save(file_name="sst-2-512/sst_2_train_data.mindrecord",
                   num_files=1,
                   file_type='mindrecord')
dev_dataset.save(file_name="sst-2-512/sst_2_test_data.mindrecord",
                   num_files=1,
                   file_type='mindrecord')

In [22]:
from mindspore import dataset
d1 = dataset.MindDataset(dataset_files="sst-2-512/sst_2_train_data.mindrecord")
d2 = dataset.MindDataset(dataset_files="sst-2-512/sst_2_test_data.mindrecord")
print(d1, d1.get_batch_size(),d1.get_dataset_size())
print(d2, d2.get_batch_size(),d2.get_dataset_size())

<mindspore.dataset.engine.datasets_standard_format.MindDataset object at 0x7fa77cf055b0> 1 67349
<mindspore.dataset.engine.datasets_standard_format.MindDataset object at 0x7fa890610bb0> 1 872


In [23]:
d1 = d1.batch(16)
d2 = d2.batch(16)
print(d1, d1.get_batch_size(),d1.get_dataset_size())
print(d2, d2.get_batch_size(),d2.get_dataset_size())

<mindspore.dataset.engine.datasets.BatchDataset object at 0x7fa773daa820> 16 4210
<mindspore.dataset.engine.datasets.BatchDataset object at 0x7fa773daa340> 16 55


In [24]:
# batch and tokenizer -> (input_ids ,token_type_ids, attention_mask, label)
print(train_dataset.get_col_names())
# for data in train_dataset_1.create_dict_iterator():
#     print(data['input_ids'].shape)
#     print(data['attention_mask'].shape)
#     print(data['token_type_ids'].shape)
#     print(data['label'].shape)
#     print(len(data))
#     break
for data in d1.create_dict_iterator():
    print(data['input_ids'].squeeze(-2).shape)
    print(data['attention_mask'].shape)
    print(data['token_type_ids'].shape)
    print(data['label'].shape)
    print(len(data))
    break

['input_ids', 'token_type_ids', 'attention_mask', 'label']
(16, 512)
(16, 1, 512)
(16, 1, 512)
(16, 1, 1)
4


In [ ]:
# test load ckpt
from src.config import BertConfig
ckpt_file = "checkpoint/bert_L4_H128_step_308749_card_id_0.ckpt"
config_file = 'config/bert_L4_H128_config.json'
config = BertConfig(config_file)
print(config.vocab_size)
model = BertForPretraining(config)

before_model = BertForPretraining(config)
dict_before = before_model.bert.parameters_dict()

dict_after = mindspore.load_checkpoint(ckpt_file)
mindspore.load_param_into_net(model, dict_after)
model = model.bert
dict_after = model.parameters_dict()
import numpy as np
list_before = list(dict_before.values())
list_after = list(dict_after.values())

print(np.allclose(list_before[2].asnumpy(), list_after[2].asnumpy(), rtol=1e-3))

In [ ]:
bert_sst_2 = BertBinaryClassificationModel(config, ckpt_file)
# test output 

for input_ids, token_type_ids, attention_mask, label in train_dataset.create_tuple_iterator():
    print(len(input_ids))
    loss, logits = bert_sst_2(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask, label=label)
    break

for item in bert_sst_2.get_parameters():
    print(item)

In [ ]:
import os
def getpwd():
    pwd = sys.path[0]
    if os.path.isfile(pwd):
        pwd = os.path.dirname(pwd)
    return pwd

In [ ]:
def train_loop(model, dataset, optimizer):
    def forward_fn(input_ids, attention_mask, token_type_ids, label):
        loss, logits = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        label=label)
        return loss, logits
    grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)

    def train_step(input_ids, attention_mask, token_type_ids, label):
        (loss, _), grads = grad_fn(input_ids, attention_mask, token_type_ids, label)
        loss = ops.depend(loss, optimizer(grads))
        return loss

    size = dataset.get_dataset_size()
    model.set_train()

    for batch, (input_ids, attention_mask, token_type_ids, label) in enumerate(dataset.create_tuple_iterator()):
        loss = train_step(input_ids, attention_mask, token_type_ids, label)
        if batch % 10 == 0:
            loss, current = loss.asnumpy(), batch
            print(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")

In [ ]:
def test_loop(model, dataset, output_file, current_step, current_epoch):
#def test_loop(model, dataset):
    num_batches = dataset.get_dataset_size()
    model.set_train(False)
    total, test_loss, correct = 0, 0, 0
    for input_ids, attention_mask, token_type_ids, label in dataset.create_tuple_iterator():
        loss, logits = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        label=label)
        total += len(input_ids)
        test_loss += loss.asnumpy()
        correct_loop = (logits.argmax(1) == label.view(-1)).asnumpy().sum()
        correct += correct_loop
    test_loss /= num_batches
    correct /= total
    if correct*100 >= 82.84:
        output_ckpt_file = os.path.join(output_file, "{epoch}_epoch_{step}_step_acc_{acc}%.ckpt".format(
            epoch=current_epoch, step=current_step, acc=(correct*100)
        ))
        mindspore.save_checkpoint(model, output_ckpt_file)
    print(f"Test: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
def train_and_test(model, train_dataset, dev_dataset, optimizer, current_epoch):
    output_file = os.path.join(getpwd(), "outputs")
    if not os.path.exists(output_file):
        try:
            os.mkdir(output_file)
        except FileExistsError:
            pass
    def forward_fn(input_ids, attention_mask, token_type_ids, label):
        loss, logits = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        label=label)
        return loss, logits
    grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)

    def train_step(input_ids, attention_mask, token_type_ids, label):
        (loss, _), grads = grad_fn(input_ids, attention_mask, token_type_ids, label)
        loss = ops.depend(loss, optimizer(grads))
        return loss

    size = train_dataset.get_dataset_size()
    model.set_train(True)

    for batch, (input_ids, attention_mask, token_type_ids, label) in enumerate(train_dataset.create_tuple_iterator()):
        loss = train_step(input_ids, attention_mask, token_type_ids, label)
        if batch % 10 == 0:
            loss, current = loss.asnumpy(), batch
            print(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")
        # add test
        if batch % 100 == 0:
            print("In epoch {} _ batch {} testing".format(current_epoch + 1, current))
            test_loop(model, dev_dataset, output_file, batch, current_epoch)
            # test_loop(model, dev_dataset)

In [ ]:
# train set
epoch_num = 10
config_file = 'config/bert_L4_H128_config.json'
ckpt_file = "checkpoint/bert_L4_H128_step_308749_card_id_0.ckpt"
config = BertConfig(config_file)
model = BertForPretraining(config)
bert_sst_2 = BertBinaryClassificationModel(config, ckpt_file)

# freeze_layer for classify task
# freeze_params = list(filter(lambda x: 'bert.encoder.layer.0' in x.name \
#                                     or 'bert.encoder.layer.1' in x.name \
#                                     or 'bert.encoder.layer.2' in x.name \
#                                     or 'bert.embeddings.token_type_embeddings' in x.name \
#                                     or 'bert.embeddings.position_embeddings' in x.name \
#                                     or 'bert.embeddings.word_embeddings' in x.name, \
#                             bert_sst_2.bert.trainable_params()))
# freeze_params = list(filter(lambda x: 'bert' in x.name, bert_sst_2.bert.trainable_params()))
# for item in freeze_params:
#     item.requires_grad = False

In [ ]:
params = bert_sst_2.trainable_params()
# print(params)

In [ ]:
# AdamWeightDecay
from src.optimization import BertLearningRate
# lr_schedule_for_backbone = BertLearningRate(learning_rate=5e-5,
#                                    end_learning_rate=0.0,
#                                    warmup_steps=int(train_dataset.get_dataset_size() * epoch_num * 0.1),
#                                    decay_steps=train_dataset.get_dataset_size() * epoch_num,
#                                    power=1.0)

lr_schedule = BertLearningRate(learning_rate=5e-5,
                                   end_learning_rate=0.0,
                                   warmup_steps=int(train_dataset.get_dataset_size() * epoch_num * 0.1),
                                   decay_steps=train_dataset.get_dataset_size() * epoch_num,
                                   power=1.0)
# decay_params = list(filter(lambda x: 'gamma' not in x.name \
#                                         and 'beta' not in x.name \
#                                         and 'bias' not in x.name, params))
# other_params = list(filter(lambda x: 'gamma' in x.name \
#                                         or 'beta' in x.name \
#                                         or 'bias' in x.name, params))
# group_params = [{'params': decay_params, 'weight_decay': 0.01},
#                     {'params': other_params},
#                     {'order_params': params}]

# classify_params = list(filter(lambda x: 'classifier' in x.name, \
#                             bert_sst_2.trainable_params()))

# other_params = list(filter(lambda x: 'classifier' not in x.name, \
#                             bert_sst_2.trainable_params()))

# group_params = [{'params': classify_params, 'weight_decay': 0.01, 'lr': lr_schedule_for_finetuing},
#                 {'params': other_params, 'lr': lr_schedule_for_finetuing}]

optimizer = nn.AdamWeightDecay(params, lr_schedule, eps=1e-8)


In [ ]:
for epoch in range(epoch_num):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train_and_test(bert_sst_2, train_dataset, dev_dataset, optimizer, epoch)
print("Done!")